In [1]:
import os
from dotenv import load_dotenv

# Load .env variables
load_dotenv()

# Get the API key
OPENROUTER_KEY = os.getenv("OPENROUTER_KEY")
if not OPENROUTER_KEY:
    raise ValueError("⚠️ Missing OPENROUTER_KEY in .env file.")

# Set environment for LangChain
os.environ["OPENAI_API_KEY"] = OPENROUTER_KEY
os.environ["OPENAI_BASE_URL"] = "https://openrouter.ai/api/v1"

print("✅ Environment and OpenRouter API configured successfully.")


✅ Environment and OpenRouter API configured successfully.


In [2]:
from langchain_openai import ChatOpenAI
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

c:\Users\Blue\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Create a message prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful and professional AI assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Initialize the OpenRouter model
llm = ChatOpenAI(
    model="gpt-4o-mini",  # change to gpt-4o for higher quality
    temperature=0.7
)

# Define session storage for memory
store = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Combine everything into a memory-aware chain
chatbot = RunnableWithMessageHistory(
    runnable = prompt | llm,
    get_session_history = get_session_history,
    input_messages_key = "input",
    history_messages_key = "history"
)

print("✅ Chatbot initialized with message-based memory.")

✅ Chatbot initialized with message-based memory.


In [4]:
session_id = "user_001"

response1 = chatbot.invoke(
    {"input": "Hello, who won the FIFA World Cup in 2022?"},
    config={"configurable": {"session_id": session_id}}
)
print("Bot:", response1.content)

response2 = chatbot.invoke(
    {"input": "Can you remind me what I just asked?"},
    config={"configurable": {"session_id": session_id}}
)
print("Bot:", response2.content)


Bot: Argentina won the FIFA World Cup in 2022. They defeated France in the final, which ended in a dramatic penalty shootout after a 3-3 draw in regulation and extra time. This victory marked Argentina's third World Cup title, with their previous wins occurring in 1978 and 1986.
Bot: You asked who won the FIFA World Cup in 2022.
